In [ ]:
%%time

import numpy as np
import pandas as pd
import pickle

test_df = pd.read_pickle('test_df.pkl')

X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')
X_test = np.load('X_test.npy')

y_train = pd.read_pickle('y_train.pkl')
y_val = pd.read_pickle('y_val.pkl')

In [ ]:
from keras import backend as K


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
 
def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2*((p*r)/(p+r+K.epsilon()))

In [ ]:
from keras.models import load_model

#model = load_model('model.h5', {'f1': f1, 'precision':precision, 'recall': recall})
#model.load_weights('weights.h5')

model = load_model('model_3_embed_cleanup_bn.h5', {'f1': f1, 'precision':precision, 'recall': recall})
model.load_weights('weights_3_embed_cleanup_bn_512x2.h5')

print(model.summary())

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', f1, recall, precision])

# Predict validation labels

In [ ]:
%%time

pred_val = model.predict([X_val], batch_size=1024, verbose=1)

# Find best threshold

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

def bestThreshold(y_true,y_pred):
    idx = 0
    cur_f1 = 0
    cur_prec = 0
    cur_recall = 0
    max_f1 = 0
    thres = 0
    for idx in np.arange(0.1, 0.501, 0.01):
        cur_f1 = f1_score(y_true, np.array(y_pred)> idx)
        cur_recall = recall_score(y_true, np.array(y_pred)> idx)
        cur_prec = precision_score(y_true, np.array(y_pred)> idx)
        print('Current threshold is {:.4f} with F1 score: {:.4f}, Recall score: {:.4f}, Precision score: {:.4f}'
              .format(idx, cur_f1, cur_recall, cur_prec)
             )
        if cur_f1 > max_f1:
            max_f1 = cur_f1
            thres = idx
    print('best threshold is {:.4f} with F1 score: {:.4f}'.format(thres, max_f1))
    return thres
threshold = bestThreshold(y_val,pred_val)

# Predict test labels

In [ ]:
%%time

pred_test = model.predict([X_test], batch_size=1024, verbose=1)

# Post-Analysis

In [ ]:
pred_df = pd.DataFrame({"qid":test_df["qid"].values, "question_text":test_df["question_text"].values})
pred_df['prediction'] = (pred_test > threshold).astype(int)

In [ ]:
import numpy as np


target_ratios = pred_df.prediction.value_counts(normalize=True)

print(target_ratios)

target_ratios.plot(kind='bar', title='Ratios (target)')

## Predicted sincere and insincere samples

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = -1

In [ ]:
pd_sincere = pred_df.loc[pred_df['prediction'] == 0].sample(100)
pd_insincere = pred_df.loc[pred_df['prediction'] == 1].sample(100)

df = pd.concat([pd_sincere, pd_insincere])
df.to_csv("predictions.csv", index=False)

In [ ]:
print("Sample insincere questions (predicted)")
pred_df.loc[pred_df['prediction'] == 1].sample(100)

In [ ]:
print("Sample sincere questions (predicted)")
pred_df.loc[pred_df['prediction'] == 0].sample(100)